In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from tableone import TableOne

import statsmodels.api as sm
from statsmodels.formula.api import mixedlm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import numpy as np

pd.set_option('future.no_silent_downcasting', True)

In [2]:
df=pd.read_excel('data/WTW & ACD Extraction Sheet.xlsx')
df = df.replace(['UK'], np.nan)

# Count of missing values

In [3]:
# count missing values
missing_counts = df[['IOLMaster_WTW','Argos_WTW', 'Schwind_WTW', 'Digital_WTW', 'Clinic_Procedure_WTW', 'IOLMaster_ACD', 'Argos_ACD ', 'Schwind_ACD', 'Pentacam_ACD','Clinic_Procedure_ACD']].isna().sum()

print(missing_counts)

IOLMaster_WTW            0
Argos_WTW               66
Schwind_WTW              0
Digital_WTW             65
Clinic_Procedure_WTW    66
IOLMaster_ACD            0
Argos_ACD               60
Schwind_ACD              0
Pentacam_ACD             0
Clinic_Procedure_ACD     0
dtype: int64


# Demographic

In [4]:
df_demo= df.drop_duplicates(subset='PHN').copy()
columns=['Age','Sex']
categorical=['Sex']
nonnormal=[]
TableOne(df_demo, columns=columns, categorical=categorical, nonnormal=nonnormal, pval=False, htest_name=False, normal_test=True, include_null=False)

Missing     Overall
n                                112
Age, mean (SD)         0  33.8 (6.9)
Sex, n (%)     F       1   63 (56.8)
               M           48 (43.2)

# White to white

In [5]:
# Format the data into long table
df_long = pd.melt(
    df,
    id_vars=["PHN", "Patient", "Year", "Age", "Sex", "Eye"], 
    value_vars=['IOLMaster_WTW','Argos_WTW','Schwind_WTW','Digital_WTW','Clinic_Procedure_WTW'],
    var_name="Device",
    value_name="WTW_Measure"
)

# Drop rows where the measurement is actually missing
df_long = df_long.dropna(subset=['WTW_Measure'])

# Define expected devices (all WTW measurement types)
devices = ['IOLMaster_WTW','Argos_WTW','Schwind_WTW','Digital_WTW','Clinic_Procedure_WTW']

# Build full index: PHN × Eye × Device
idx = pd.MultiIndex.from_product(
    [df_long['PHN'].unique(), ['OD','OS'], devices],
    names=['PHN','Eye','Device']
)

# Reindex so that missing combinations appear as NaN
df_long = (
    df_long.set_index(['PHN','Eye','Device'])
    .reindex(idx)
    .reset_index()
)

# Now missing OD/OS/device combos show up as NaN in WTW_Measure
display(df_long.head(10))

,PHN,Eye,Device,Patient,Year,Age,Sex,WTW_Measure
0,847462010,OD,IOLMaster_WTW,2010.0,2023.0,41.0,M,11.90000
1,847462010,OD,Argos_WTW,2010.0,2023.0,41.0,M,12.05000
2,847462010,OD,Schwind_WTW,2010.0,2023.0,41.0,M,12.07000
3,847462010,OD,Digital_WTW,2010.0,2023.0,41.0,M,11.65000
4,847462010,OD,Clinic_Procedure_WTW,2010.0,2023.0,41.0,M,11.63125
5,847462010,OS,IOLMaster_WTW,2010.0,2023.0,41.0,M,11.90000
6,847462010,OS,Argos_WTW,2010.0,2023.0,41.0,M,12.06000
7,847462010,OS,Schwind_WTW,2010.0,2023.0,41.0,M,12.17000
8,847462010,OS,Digital_WTW,2010.0,2023.0,41.0,M,11.65000
9,847462010,OS,Clinic_Procedure_WTW,2010.0,2023.0,41.0,M,11.63625


In [6]:
columns=['WTW_Measure']
groupby='Device'
categorical=[]
nonnormal=[]

In [7]:
# Both Eyes
display(TableOne(df_long, columns=columns, categorical=categorical, nonnormal=nonnormal, groupby=groupby, pval=True, htest_name=True, normal_test=True, include_null=False))

df_long = df_long.dropna(subset=['WTW_Measure'])

# Tukey HSD
tukey = pairwise_tukeyhsd(
    endog=df_long['WTW_Measure'].values,
    groups=df_long['Device'].values,
    alpha=0.05
)
print(tukey.summary())

Grouped by Device                                                                                                          
                                  Missing     Overall   Argos_WTW Clinic_Procedure_WTW Digital_WTW IOLMaster_WTW Schwind_WTW P-Value           Test
n                                                1120         224                  224         224           224         224                       
WTW_Measure, mean (SD)                197  12.0 (0.4)  12.0 (0.4)           11.7 (0.3)  11.8 (0.4)    12.2 (0.4)  12.1 (0.4)  <0.001  One-way ANOVA

              Multiple Comparison of Means - Tukey HSD, FWER=0.05               
       group1               group2        meandiff p-adj   lower   upper  reject
--------------------------------------------------------------------------------
           Argos_WTW Clinic_Procedure_WTW  -0.3093    0.0 -0.4202 -0.1984   True
           Argos_WTW          Digital_WTW  -0.2273    0.0 -0.3381 -0.1166   True
           Argos_WTW        IOLMaster_WTW   0.1191 0.0132  0.0167  0.2215   True
           Argos_WTW          Schwind_WTW   0.0814 0.1909  -0.021  0.1838  False
Clinic_Procedure_WTW          Digital_WTW   0.0819 0.2558 -0.0288  0.1927  False
Clinic_Procedure_WTW        IOLMaster_WTW   0.4284    0.0   0.326  0.5308   True
Clinic_Procedure_WTW          Schwind_WTW   0.3907    0.0  0.2883  0.4931   True
         Digital_WTW        IOLMaster_WTW   0.3464    0.0  0.2442  0.4486   True
         Digital_WTW          Schwind_WTW   0.3088    0.0  0.2065   0.411   True
       IOLMaster_WTW        

In [8]:
#------------- Each eye seperately --------------
df_OD= df_long[df_long['Eye']=='OD']
df_OS= df_long[df_long['Eye']=='OS']
#-------------------------------------------------

print ('OD')
display(TableOne(df_OD, columns=columns, categorical=categorical, nonnormal=nonnormal, groupby=groupby, pval=True, htest_name=True, normal_test=True, include_null=False))

df_OD = df_OD.dropna(subset=['WTW_Measure'])
tukey = pairwise_tukeyhsd(
    endog=df_OD['WTW_Measure'].values,
    groups=df_OD['Device'].values,
    alpha=0.05
)
print('OD Post-hoc\n',tukey.summary())



print ('OS')
display(TableOne(df_OS, columns=columns, categorical=categorical, nonnormal=nonnormal, groupby=groupby, pval=True, htest_name=True, normal_test=True, include_null=False))
#------------- POST-HOC --------------

# Filter to OD, drop NaNs
df_OS = df_OS.dropna(subset=['WTW_Measure'])

# Tukey HSD
tukey = pairwise_tukeyhsd(
    endog=df_OS['WTW_Measure'].values,
    groups=df_OS['Device'].values,
    alpha=0.05
)

print('OS Post-hoc\n',tukey.summary())

OD


Grouped by Device                                                                                                          
                                  Missing     Overall   Argos_WTW Clinic_Procedure_WTW Digital_WTW IOLMaster_WTW Schwind_WTW P-Value           Test
n                                                 461          79                   79          79           112         112                       
WTW_Measure, mean (SD)                  0  12.0 (0.4)  12.0 (0.4)           11.7 (0.3)  11.8 (0.4)    12.1 (0.4)  12.1 (0.4)  <0.001  One-way ANOVA

OD Post-hoc
               Multiple Comparison of Means - Tukey HSD, FWER=0.05               
       group1               group2        meandiff p-adj   lower   upper  reject
--------------------------------------------------------------------------------
           Argos_WTW Clinic_Procedure_WTW  -0.3187    0.0 -0.4757 -0.1616   True
           Argos_WTW          Digital_WTW  -0.2329 0.0005   -0.39 -0.0759   True
           Argos_WTW        IOLMaster_WTW    0.107 0.2578  -0.038   0.252  False
           Argos_WTW          Schwind_WTW   0.0816 0.5359 -0.0634  0.2266  False
Clinic_Procedure_WTW          Digital_WTW   0.0858 0.5657 -0.0713  0.2428  False
Clinic_Procedure_WTW        IOLMaster_WTW   0.4257    0.0  0.2807  0.5707   True
Clinic_Procedure_WTW          Schwind_WTW   0.4003    0.0  0.2553  0.5453   True
         Digital_WTW        IOLMaster_WTW   0.3399    0.0  0.1949  0.4849   True
         Digital_WTW          Schwind_WTW   0.3145    0.0  0.1695  0.4596   True
       IOLMaste

Grouped by Device                                                                                                          
                                  Missing     Overall   Argos_WTW Clinic_Procedure_WTW Digital_WTW IOLMaster_WTW Schwind_WTW P-Value           Test
n                                                 462          79                   79          80           112         112                       
WTW_Measure, mean (SD)                  0  12.0 (0.4)  12.0 (0.4)           11.7 (0.4)  11.8 (0.4)    12.2 (0.4)  12.1 (0.4)  <0.001  One-way ANOVA

OS Post-hoc
               Multiple Comparison of Means - Tukey HSD, FWER=0.05               
       group1               group2        meandiff p-adj   lower   upper  reject
--------------------------------------------------------------------------------
           Argos_WTW Clinic_Procedure_WTW  -0.2999    0.0 -0.4579 -0.1419   True
           Argos_WTW          Digital_WTW  -0.2219 0.0012 -0.3794 -0.0644   True
           Argos_WTW        IOLMaster_WTW   0.1312 0.1011 -0.0147  0.2771  False
           Argos_WTW          Schwind_WTW   0.0812 0.5473 -0.0647  0.2271  False
Clinic_Procedure_WTW          Digital_WTW    0.078 0.6559 -0.0795  0.2355  False
Clinic_Procedure_WTW        IOLMaster_WTW   0.4311    0.0  0.2852   0.577   True
Clinic_Procedure_WTW          Schwind_WTW   0.3811    0.0  0.2352   0.527   True
         Digital_WTW        IOLMaster_WTW   0.3531    0.0  0.2077  0.4984   True
         Digital_WTW          Schwind_WTW   0.3031    0.0  0.1577  0.4484   True
       IOLMaste

In [9]:
#------------- Mixed Model --------------
df_long['PHN'] = df_long['PHN'].astype(str)
df_model = df_long.dropna(subset=['WTW_Measure'])

# Now fit the mixed model
model = mixedlm("WTW_Measure ~ Device + Eye", df_model, groups=df_model["PHN"])
result = model.fit()
print(result.summary())

                  Mixed Linear Model Regression Results
Model:                   MixedLM      Dependent Variable:      WTW_Measure
No. Observations:        923          Method:                  REML       
No. Groups:              112          Scale:                   0.0133     
Min. group size:         6            Log-Likelihood:          424.7151   
Max. group size:         10           Converged:               Yes        
Mean group size:         8.2                                              
--------------------------------------------------------------------------
                               Coef.  Std.Err.    z    P>|z| [0.025 0.975]
--------------------------------------------------------------------------
Intercept                      12.072    0.034 349.925 0.000 12.004 12.140
Device[T.Clinic_Procedure_WTW] -0.309    0.013 -23.839 0.000 -0.335 -0.284
Device[T.Digital_WTW]          -0.329    0.014 -23.773 0.000 -0.356 -0.302
Device[T.IOLMaster_WTW]         0.077    0.0

# ACD

In [10]:
# Format the data into long table
df_long = pd.melt(
    df,
    id_vars=["PHN", "Patient", "Year", "Age", "Sex", "Eye"],  # keep these
    value_vars=['IOLMaster_ACD', 'Argos_ACD ', 'Schwind_ACD', 'Pentacam_ACD','Clinic_Procedure_ACD'],
    var_name="Device",
    value_name="ACD_Measure"
)

# Drop rows where the measurement is actually missing
df_long = df_long.dropna(subset=['ACD_Measure'])

# Define expected devices (all WTW measurement types)
devices = ['IOLMaster_ACD', 'Argos_ACD ', 'Schwind_ACD', 'Pentacam_ACD','Clinic_Procedure_ACD']

# Build full index: PHN × Eye × Device
idx = pd.MultiIndex.from_product(
    [df_long['PHN'].unique(), ['OD','OS'], devices],
    names=['PHN','Eye','Device']
)

# Reindex so that missing combinations appear as NaN
df_long = (
    df_long.set_index(['PHN','Eye','Device'])
    .reindex(idx)
    .reset_index()
)

# Now missing OD/OS/device combos show up as NaN in ACD_Measure
display(df_long.head(10))

,PHN,Eye,Device,Patient,Year,Age,Sex,ACD_Measure
0,847462010,OD,IOLMaster_ACD,2010.0,2023.0,41.0,M,3.650
1,847462010,OD,Argos_ACD,2010.0,2023.0,41.0,M,3.740
2,847462010,OD,Schwind_ACD,2010.0,2023.0,41.0,M,3.310
3,847462010,OD,Pentacam_ACD,2010.0,2023.0,41.0,M,3.200
4,847462010,OD,Clinic_Procedure_ACD,2010.0,2023.0,41.0,M,3.255
5,847462010,OS,IOLMaster_ACD,2010.0,2023.0,41.0,M,3.610
6,847462010,OS,Argos_ACD,2010.0,2023.0,41.0,M,3.680
7,847462010,OS,Schwind_ACD,2010.0,2023.0,41.0,M,3.220
8,847462010,OS,Pentacam_ACD,2010.0,2023.0,41.0,M,3.170
9,847462010,OS,Clinic_Procedure_ACD,2010.0,2023.0,41.0,M,3.195


In [11]:
columns=['ACD_Measure']
groupby='Device'
categorical=[]
nonnormal=['ACD_Measure']

In [12]:
# Both Eyes
display(TableOne(df_long, columns=columns, categorical=categorical, nonnormal=nonnormal, groupby=groupby, pval=True, htest_name=True, normal_test=True, include_null=False))

df_long = df_long.dropna(subset=['ACD_Measure'])

# Tukey HSD
tukey = pairwise_tukeyhsd(
    endog=df_long['ACD_Measure'].values,
    groups=df_long['Device'].values,
    alpha=0.05
)
print(tukey.summary())

Grouped by Device                                                                                                                        
                                       Missing        Overall     Argos_ACD  Clinic_Procedure_ACD  IOLMaster_ACD   Pentacam_ACD    Schwind_ACD P-Value            Test
n                                                        1120            224                  224            224            224            224                        
ACD_Measure, median [Q1,Q3]                 60  3.4 [3.1,3.6]  3.8 [3.6,4.0]        3.2 [3.1,3.4]  3.7 [3.5,3.9]  3.2 [3.0,3.4]  3.2 [3.1,3.4]  <0.001  Kruskal-Wallis
[1] Normality test reports non-normal
                                    distributions for: ACD_Measure.

              Multiple Comparison of Means - Tukey HSD, FWER=0.05               
       group1               group2        meandiff p-adj   lower   upper  reject
--------------------------------------------------------------------------------
          Argos_ACD  Clinic_Procedure_ACD  -0.5803    0.0 -0.6493 -0.5114   True
          Argos_ACD         IOLMaster_ACD  -0.1105 0.0001 -0.1795 -0.0416   True
          Argos_ACD          Pentacam_ACD  -0.5895    0.0 -0.6584 -0.5205   True
          Argos_ACD           Schwind_ACD  -0.5712    0.0 -0.6401 -0.5022   True
Clinic_Procedure_ACD        IOLMaster_ACD   0.4698    0.0  0.4064  0.5332   True
Clinic_Procedure_ACD         Pentacam_ACD  -0.0092 0.9949 -0.0725  0.0542  False
Clinic_Procedure_ACD          Schwind_ACD   0.0092 0.9949 -0.0542  0.0725  False
       IOLMaster_ACD         Pentacam_ACD   -0.479    0.0 -0.5424 -0.4156   True
       IOLMaster_ACD          Schwind_ACD  -0.4607    0.0 -0.5241 -0.3973   True
        Pentacam_ACD        

In [13]:
#------------- Each eye seperately --------------
df_OD= df_long[df_long['Eye']=='OD']
df_OS= df_long[df_long['Eye']=='OS']
#-------------------------------------------------

print ('OD')
display(TableOne(df_OD, columns=columns, categorical=categorical, nonnormal=nonnormal, groupby=groupby, pval=True, htest_name=True, normal_test=True, include_null=False))

tukey = pairwise_tukeyhsd(
    endog=df_OD['ACD_Measure'].values,
    groups=df_OD['Device'].values,
    alpha=0.05
)

print('OD Post-hoc\n',tukey.summary())

print ('OS')
display(TableOne(df_OS, columns=columns, categorical=categorical, nonnormal=nonnormal, groupby=groupby, pval=True, htest_name=True, normal_test=True, include_null=False))




# Filter to OD, drop NaNs
df_OS = df_OS.dropna(subset=['ACD_Measure'])

# Tukey HSD
tukey = pairwise_tukeyhsd(
    endog=df_OS['ACD_Measure'].values,
    groups=df_OS['Device'].values,
    alpha=0.05
)

print('OS Post-hoc\n',tukey.summary())


OD


Grouped by Device                                                                                                                        
                                       Missing        Overall     Argos_ACD  Clinic_Procedure_ACD  IOLMaster_ACD   Pentacam_ACD    Schwind_ACD P-Value            Test
n                                                         530             82                  112            112            112            112                        
ACD_Measure, median [Q1,Q3]                  0  3.4 [3.1,3.6]  3.8 [3.6,4.0]        3.2 [3.1,3.4]  3.6 [3.5,3.9]  3.2 [3.0,3.4]  3.2 [3.1,3.4]  <0.001  Kruskal-Wallis

OD Post-hoc
               Multiple Comparison of Means - Tukey HSD, FWER=0.05               
       group1               group2        meandiff p-adj   lower   upper  reject
--------------------------------------------------------------------------------
          Argos_ACD  Clinic_Procedure_ACD  -0.5829    0.0  -0.677 -0.4888   True
          Argos_ACD         IOLMaster_ACD  -0.1131 0.0094 -0.2072  -0.019   True
          Argos_ACD          Pentacam_ACD  -0.5924    0.0 -0.6865 -0.4983   True
          Argos_ACD           Schwind_ACD  -0.5734    0.0 -0.6675 -0.4793   True
Clinic_Procedure_ACD        IOLMaster_ACD   0.4698    0.0  0.3832  0.5563   True
Clinic_Procedure_ACD         Pentacam_ACD  -0.0095 0.9982  -0.096   0.077  False
Clinic_Procedure_ACD          Schwind_ACD   0.0095 0.9982  -0.077   0.096  False
       IOLMaster_ACD         Pentacam_ACD  -0.4793    0.0 -0.5658 -0.3928   True
       IOLMaster_ACD          Schwind_ACD  -0.4603    0.0 -0.5468 -0.3737   True
        Pentaca

Grouped by Device                                                                                                                        
                                       Missing        Overall     Argos_ACD  Clinic_Procedure_ACD  IOLMaster_ACD   Pentacam_ACD    Schwind_ACD P-Value            Test
n                                                         530             82                  112            112            112            112                        
ACD_Measure, median [Q1,Q3]                  0  3.4 [3.1,3.7]  3.8 [3.6,4.0]        3.2 [3.0,3.4]  3.7 [3.5,3.9]  3.2 [3.0,3.4]  3.2 [3.1,3.4]  <0.001  Kruskal-Wallis
[1] Normality test reports non-normal
                                    distributions for: ACD_Measure.

OS Post-hoc
               Multiple Comparison of Means - Tukey HSD, FWER=0.05               
       group1               group2        meandiff p-adj   lower   upper  reject
--------------------------------------------------------------------------------
          Argos_ACD  Clinic_Procedure_ACD  -0.5777    0.0 -0.6793 -0.4762   True
          Argos_ACD         IOLMaster_ACD  -0.1079 0.0309 -0.2094 -0.0064   True
          Argos_ACD          Pentacam_ACD  -0.5865    0.0 -0.6881  -0.485   True
          Argos_ACD           Schwind_ACD   -0.569    0.0 -0.6705 -0.4674   True
Clinic_Procedure_ACD        IOLMaster_ACD   0.4699    0.0  0.3765  0.5632   True
Clinic_Procedure_ACD         Pentacam_ACD  -0.0088  0.999 -0.1021  0.0845  False
Clinic_Procedure_ACD          Schwind_ACD   0.0088  0.999 -0.0845  0.1021  False
       IOLMaster_ACD         Pentacam_ACD  -0.4787    0.0  -0.572 -0.3853   True
       IOLMaster_ACD          Schwind_ACD  -0.4611    0.0 -0.5544 -0.3677   True
        Pentaca

In [14]:
#------------- Mixed Model --------------
df_long['PHN'] = df_long['PHN'].astype(str)
df_model = df_long.dropna(subset=['ACD_Measure'])

# Now fit the mixed model
model = mixedlm("ACD_Measure ~ Device + Eye", df_model, groups=df_model["PHN"])
result = model.fit()
print(result.summary())

                  Mixed Linear Model Regression Results
Model:                   MixedLM      Dependent Variable:      ACD_Measure
No. Observations:        1060         Method:                  REML       
No. Groups:              112          Scale:                   0.0037     
Min. group size:         8            Log-Likelihood:          1168.9286  
Max. group size:         10           Converged:               Yes        
Mean group size:         9.5                                              
--------------------------------------------------------------------------
                               Coef.  Std.Err.    z    P>|z| [0.025 0.975]
--------------------------------------------------------------------------
Intercept                       3.804    0.023 165.257 0.000  3.758  3.849
Device[T.Clinic_Procedure_ACD] -0.572    0.006 -90.350 0.000 -0.584 -0.560
Device[T.IOLMaster_ACD]        -0.102    0.006 -16.151 0.000 -0.115 -0.090
Device[T.Pentacam_ACD]         -0.581    0.0

# Lenses

In [15]:
# LENSES
lenses={12.1:1, 
        12.6:2, 
        13.2:3, 
        13.7:4, 
        'Unavailable ':5}

cols_to_replace = [
   'Actual_ICL_Length_Used','IOLMaster_WTW_Order ', 'Argos_WTW_Order', 'Schwind_WTW_Order',
       'Digital_WTW_Order ', 'Clinic_Procedure_WTW_Order',
       'IOLMaster_ACD_Order', 'Argos_ACD_Order', 'Schwind_ACD_Order',
       'Pentacam_ACD_Order', 'Clinic_Procedure_ACD_Order'
]


df_lenses = df[cols_to_replace]

######### REMOVED THE ROW WHERE ALL ORDERS WERE UNAVAILABLE
df_lenses = df_lenses[~(df_lenses[['IOLMaster_WTW_Order ','Schwind_WTW_Order','Digital_WTW_Order ','IOLMaster_ACD_Order','Schwind_ACD_Order','Pentacam_ACD_Order','Clinic_Procedure_ACD_Order']] == 'Unavailable ').all(axis=1)]


df_lenses= df_lenses.replace(lenses)

###### REPLACED BY LENSE SIZES
display(df_lenses.head(10))

,Actual_ICL_Length_Used,IOLMaster_WTW_Order,Argos_WTW_Order,Schwind_WTW_Order,Digital_WTW_Order,Clinic_Procedure_WTW_Order,IOLMaster_ACD_Order,Argos_ACD_Order,Schwind_ACD_Order,Pentacam_ACD_Order,Clinic_Procedure_ACD_Order
0,2.0,3,3,3,3,2.0,3,3.0,2,2,2
1,2.0,3,3,3,3,2.0,3,3.0,2,2,2
2,3.0,5,NaN,5,3,NaN,3,NaN,3,3,3
4,1.0,2,2,2,NaN,1.0,1,1.0,1,1,1
5,1.0,2,2,2,NaN,1.0,1,1.0,1,1,1
6,2.0,3,NaN,3,2,NaN,2,NaN,2,2,2
7,2.0,3,NaN,3,2,NaN,2,NaN,2,2,2
8,2.0,3,3,3,2,2.0,2,3.0,2,2,2
9,2.0,3,3,3,2,2.0,2,3.0,2,2,2
10,3.0,4,4,3,3,3.0,3,3.0,3,3,3


In [16]:
df_lenses.columns = df_lenses.columns.str.strip()

# columns to subtract from Actual_ICL_Length_Used
order_cols = [
    'IOLMaster_WTW_Order', 'Argos_WTW_Order', 'Schwind_WTW_Order',
    'Digital_WTW_Order', 'Clinic_Procedure_WTW_Order',
    'IOLMaster_ACD_Order', 'Argos_ACD_Order', 'Schwind_ACD_Order',
    'Pentacam_ACD_Order', 'Clinic_Procedure_ACD_Order'
]


df_lenses['Actual_ICL_Length_Used'] = pd.to_numeric(df_lenses['Actual_ICL_Length_Used'], errors='coerce')

for col in order_cols:
    df_lenses[col] = pd.to_numeric(df_lenses[col], errors='coerce')  
    diff_col = f"{col}_diff"
    df_lenses[diff_col] = df_lenses[col] - df_lenses['Actual_ICL_Length_Used']


In [17]:
# Just to show how it looks
df_lenses[['Actual_ICL_Length_Used','IOLMaster_WTW_Order','IOLMaster_WTW_Order_diff']]

,Actual_ICL_Length_Used,IOLMaster_WTW_Order,IOLMaster_WTW_Order_diff
0,2.0,3,1.0
1,2.0,3,1.0
2,3.0,5,2.0
4,1.0,2,1.0
5,1.0,2,1.0
...,...,...,...
219,2.0,2,0.0
220,3.0,4,1.0
221,4.0,4,0.0
222,3.0,4,1.0


In [18]:
diff_cols = [col for col in df_lenses.columns if col.endswith('_diff')]
for col in diff_cols:
    counts = df_lenses[col].value_counts(dropna=False)  # include NaN
    percent = df_lenses[col].value_counts(normalize=True, dropna=False) * 100
    summary = pd.DataFrame({'Count': counts, 'Percent': percent})
    print(f"\n{col}:\n")
    display(summary)



IOLMaster_WTW_Order_diff:



,Count,Percent
IOLMaster_WTW_Order_diff,,
1.0,125,56.053812
0.0,86,38.565022
NaN,8,3.587444
2.0,3,1.345291
3.0,1,0.448430



Argos_WTW_Order_diff:



,Count,Percent
Argos_WTW_Order_diff,,
0.0,76,34.080717
1.0,75,33.632287
NaN,68,30.493274
-1.0,3,1.345291
2.0,1,0.448430



Schwind_WTW_Order_diff:



...

,Count,Percent
Schwind_ACD_Order_diff,,
0.0,180,80.717489
1.0,21,9.417040
-1.0,13,5.829596
NaN,8,3.587444
2.0,1,0.448430



Pentacam_ACD_Order_diff:



,Count,Percent
Pentacam_ACD_Order_diff,,
0.0,184,82.511211
1.0,17,7.623318
-1.0,13,5.829596
NaN,8,3.587444
2.0,1,0.448430



Clinic_Procedure_ACD_Order_diff:



,Count,Percent
Clinic_Procedure_ACD_Order_diff,,
0.0,183,82.062780
1.0,18,8.071749
-1.0,13,5.829596
NaN,8,3.587444
2.0,1,0.448430
